In [ ]:
# Import Packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
!pip install dash dash-bootstrap-components jupyter-dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

In [15]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Define Folder Path

folder_path = '/content/drive/My Drive/SeekOps/Flaring_Draft/'

# Load Data

combined_tables= pd.read_csv(f'{folder_path}Combined_Tables.csv')

In [4]:
# Create a Dash application:
if 'app' not in globals():
    app = JupyterDash(__name__)

    # Define the layout of the app
    app.layout = html.Div([
        dcc.Dropdown(
            id='survey-dropdown',
            options=[{'label': code, 'value': code} for code in combined_tables['Data Reference Code'].unique()],
            value=combined_tables['Data Reference Code'].unique()[0],
            clearable=False
        ),
        html.Div(id='data-table'),
        dcc.Graph(id='map')
    ])

    @app.callback(
        [Output('data-table', 'children'),
         Output('map', 'figure')],
        [Input('survey-dropdown', 'value')]
    )
    def update_dashboard(selected_code):
        # Filter data based on the selected survey code
        filtered_data = combined_tables[combined_tables['Data Reference Code'] == selected_code]

        # Define columns to display in the table (excluding 'Data Reference Code', 'Lat', and 'Lon')
        display_columns = [col for col in filtered_data.columns if col not in ['Data Reference Code', 'Lat', 'Lon']]

        # Create data table excluding 'Data Reference Code' with centered values
        table = html.Table([
            html.Thead(html.Tr([html.Th(col, style={'text-align': 'center'}) for col in display_columns])),
            html.Tbody([html.Tr([html.Td(filtered_data.iloc[0][col], style={'text-align': 'center'}) for col in display_columns])])
        ], style={'width': '100%', 'margin': 'auto'})

        # Create map with adjusted layout to remove white border
        fig = px.scatter_mapbox(
            filtered_data, lat='Lat', lon='Lon', zoom=10,
            mapbox_style='open-street-map'  # Change to 'satellite' for satellite view
        )
        fig.update_layout(
            margin={"r":0, "t":0, "l":0, "b":0}  # Removes default margins
        )

        return table, fig

    # Run the app in external mode
    app.run_server(mode='external')


/usr/local/lib/python3.10/dist-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

Dash app running on:


<IPython.core.display.Javascript object>